# Question answering (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
#managing datasets for  (NLP)
#custom package or script for evaluating NLP models
!pip install datasets evaluate transformers[sentencepiece]
#elp speed up the training of deep learning models
!pip install accelerate
#It installs Git Large File Storage (LFS),
!apt install git-lfs
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 33.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 3.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-m

You will need to setup git, adapt your email and name in the following cell.

In [ ]:
!git config --global user.email "slamtm608@gmail.com"
!git config --global user.name "MohamedMousse"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset
import datasets

In [2]:
#Import packages
import pandas as pd
import gensim


In [5]:
df = pd.read_csv('/content/train.csv')
df.head()

,item_id,domain,nn_mod,nn_asp,query_mod,query_asp,q_review_id,q_reviews_id,question,question_subj_level,ques_subj_score,is_ques_subjective,review_id,review,human_ans_spans,human_ans_indices,answer_subj_level,ans_subj_score,is_ans_subjective
0,B00BVMXBDO,movies,addictive,show,full,series,d9a9615d45df2f6e6108db4ca46bfded,399f1046fe6bd97990107f9d7aa86f4a,Who is the author of this series?,1,0.0,False,090671369dddfeb02db9bf7125a47c79,Whether it be in her portrayal of a nerdy lesb...,ANSWERNOTFOUND,"(251, 265)",1,0.000,False
1,1404918051,movies,enough simple,film,charming,movie,06ffe37a8023636a3ce00b020a517e87,42d9dd5b0c67150cac1e13308811cbb5,Can we enjoy the movie along with our family ?,1,0.5,False,a29821121e74d319cb93f77101e99c88,"An outstanding romantic comedy, 13 Going on 30...",ANSWERNOTFOUND,"(1195, 1209)",1,0.000,False
2,B0000633ZP,movies,weak,plot,bad,one,3b625c68e91b9e6987a08b84a9a9d234,32d06ccf2132cda644aea791fa688c53,Does this one good?,5,0.6,True,12a1b821f761bd19a75be7b16cef4a7c,"To let the truth be known, I watched this movi...",ANSWERNOTFOUND,"(1476, 1490)",5,0.000,False
3,B0000AQS0F,movies,outstanding,show,wonderful,series,f3abfa98b011127e7cb49bcd07f8deeb,e546636f0bb9f93d5f24b4ade9ebab45,Is this series good and excelent?,1,0.6,True,cd0f92322e67cc9d70de6674caace78c,"At the time of my review, there had been 910 c...",this show is OUTSTANDING,"(296, 320)",1,0.875,True
4,B003Y5H5FG,movies,great,production design,great,costume design,1b03744e764b257592c2c768345c14bc,a0a97e460a194bcb3286fe68d20aadc2,How is the costume design?,1,0.0,False,f6b5024393ebc70287befdaf47a50b75,"""Fright Night"" is great! This is how the story...",The costume design by Susan Matheson is great,"(1254, 1299)",1,0.750,True


In [6]:

docs = df.review.apply(gensim.utils.simple_preprocess)

In [7]:
#Word2vec implementation
model = gensim.models.Word2Vec(vector_size=100,  # Dimensionality of the word vectors
                          window=5,         # Use a window of size 5 i.e. 5 words before the present word and 5 words ahead.
                          sg=1,             # Skip-Gram model (1 for Skip-Gram, 0 for CBOW)
                          min_count=1,      # Ignores all words with a total frequency lower than this
                          workers=4)        # Number of CPU cores to use for training the model

In [8]:
#Analyze the provided documents to identify unique words (tokens) and assign them unique integer IDs.
model.build_vocab(docs)

In [9]:
model.epochs

5

In [10]:
model.corpus_count

2501

In [11]:
model.train(docs,total_examples=model.corpus_count,epochs=model.epochs)

(3102174, 4033970)

In [12]:
model.wv['great']

array([-0.7530584 , -0.36696687,  0.5286425 , -0.5964628 ,  0.5840985 ,
       -0.27999282,  0.23042016, -0.21398556, -0.5222285 ,  0.33289123,
        0.07756665, -0.3412642 ,  0.4953337 ,  0.11334932,  0.36645433,
       -0.14855722,  0.0901303 , -0.5359254 , -0.64336014, -0.18892784,
        0.43206736, -0.12063006,  0.61730784,  0.06705844,  0.32707173,
       -0.21968715,  0.43942323, -0.13562508, -0.4146467 , -0.01367013,
        0.832996  ,  0.06326632,  0.08273792,  0.05049942,  0.04457578,
        0.10458423, -0.39072728, -0.28972322,  0.11371019, -0.47244754,
        0.36135843, -0.9513618 , -0.58261746, -0.6712056 , -0.25406888,
       -0.51942194, -0.09854702,  0.12807947,  0.07726268, -0.4777658 ,
        0.54805404, -0.14904194, -0.3000238 , -0.04299494,  0.13737932,
       -0.49484062,  0.76358557,  0.472512  , -0.4022879 , -0.10500596,
       -0.21619709,  0.14561369,  0.23861906,  0.16090119,  0.26942265,
        0.50154   , -0.05278317,  0.7806742 , -0.64076   ,  0.54

In [13]:
#find similar words to the given word
model.wv.most_similar('bad')

[('good', 0.7066650986671448),
 ('outweigh', 0.6750352382659912),
 ('hokey', 0.66383296251297),
 ('necessarily', 0.6571234464645386),
 ('vulgar', 0.656567394733429),
 ('embarrassing', 0.6509606242179871),
 ('unsympathetic', 0.64357990026474),
 ('horrible', 0.6412188410758972),
 ('laughable', 0.6394578814506531),
 ('decent', 0.6379543542861938)]

In [15]:
#words which doesn't match
model.wv.doesnt_match('house rent trust apartment'.split())

'house'

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
tokenizer.is_fast

True

In [ ]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
import pandas as pd
df_train=pd.read_csv('/content/train.csv')
df_test=pd.read_csv('/content/test.csv')

In [ ]:
df_train.columns

Index(['item_id', 'domain', 'nn_mod', 'nn_asp', 'query_mod', 'query_asp',
       'q_review_id', 'q_reviews_id', 'question', 'question_subj_level',
       'ques_subj_score', 'is_ques_subjective', 'review_id', 'review',
       'human_ans_spans', 'human_ans_indices', 'answer_subj_level',
       'ans_subj_score', 'is_ans_subjective'],
      dtype='object')

In [ ]:
df_train.iloc[0].question

'Who is the author of this series?'

In [ ]:
df_train.iloc[0].review

"Whether it be in her portrayal of a nerdy lesbian or a punk rock rebel, Maslany's plural personalities, (though very stereotypical), are entertaining eye-candy. Combined with a complex and unpredictable plot line, this show is surprisingly addictive. ANSWERNOTFOUND"

In [ ]:
df_train.iloc[0].human_ans_indices

'(251, 265)'

In [ ]:
df_train.iloc[0].review[251:265]

'ANSWERNOTFOUND'

In [ ]:
df_train=df_train[['question','human_ans_indices','review','human_ans_spans']]
df_test=df_test[['question','human_ans_indices','review','human_ans_spans']]

In [ ]:
import numpy as np
df_train['id']=np.linspace(0,len(df_train)-1,len(df_train))
df_test['id']=np.linspace(0,len(df_test)-1,len(df_test))

df_train['id']=df_train['id'].astype(str)
df_test['id']=df_test['id'].astype(str)

In [ ]:
int(df_train.iloc[0].human_ans_indices.split('(')[1].split(',')[0])

251

In [ ]:
float(df_train.iloc[0].human_ans_indices.split('(')[1].split(',')[1].split(' ')[1].split(')')[0])

265.0

In [ ]:
df_train['answers']=df_train['human_ans_spans']
df_test['answers']=df_test['human_ans_spans']

In [ ]:
for i in range(0,len(df_train)):
  # Initialize an empty dictionary for each row

  answer1={}
  # Extract start index (si) from 'human_ans_indices' column and convert to integer
  si=int(df_train.iloc[i].human_ans_indices.split('(')[1].split(',')[0])
   # Extract end index (ei) from 'human_ans_indices' column and convert to integer
  ei=int(df_train.iloc[i].human_ans_indices.split('(')[1].split(',')[1].split(' ')[1].split(')')[0])
  # Extract text based on start and end indices from 'review' column and assign to 'text' key in answer1
  answer1['text']=[df_train.iloc[i].review[si:ei]]
  # Assign start index to 'answer_start' key in answer1
  answer1['answer_start']=[si]
  # Assign the constructed answer1 dictionary to the 'answers' column in the dataframe at row i
  df_train.at[i, 'answers']=answer1
  #print(df_train.iloc[i].answers,df_train.iloc[i].human_ans_spans)

In [ ]:
for i in range(0,len(df_test)):
  answer1={}
  si=int(df_test.iloc[i].human_ans_indices.split('(')[1].split(',')[0])
  ei=int(df_test.iloc[i].human_ans_indices.split('(')[1].split(',')[1].split(' ')[1].split(')')[0])
  answer1['text']=[df_test.iloc[i].review[si:ei]]
  answer1['answer_start']=[si]
  df_test.at[i, 'answers']=answer1
  #print(df_train.iloc[i].answers,df_train.iloc[i].human_ans_spans)

In [ ]:
df_train.columns

Index(['question', 'human_ans_indices', 'review', 'human_ans_spans', 'id',
       'answers'],
      dtype='object')

In [ ]:
# Rename columns of training dataframe
df_train.columns=['question', 'human_ans_indices', 'context', 'human_ans_spans', 'id',
       'answers']
# Rename columns of testing dataframe
df_test.columns=['question', 'human_ans_indices', 'context', 'human_ans_spans','id',
       'answers']

In [ ]:
# Convert testing dataframe to dataset
val_dataset2 = datasets.Dataset.from_pandas(df_test)
# Convert training dataframe to dataset
train_dataset2 = datasets.Dataset.from_pandas(df_train)


In [ ]:
train_dataset = train_dataset2.map(
    preprocess_training_examples,
    #This indicates that the mapping function should be applied to examples in batches rather than individually for efficiency.
    batched=True,
    remove_columns=train_dataset2.column_names,
)
len(train_dataset2), len(train_dataset)

Map:   0%|          | 0/2501 [00:00<?, ? examples/s]

(2501, 4862)

In [ ]:
train_dataset2.shape

(2501, 6)

In [ ]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
validation_dataset = val_dataset2.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=val_dataset2.column_names,
)
len(validation_dataset)

Map:   0%|          | 0/582 [00:00<?, ? examples/s]

1104

In [ ]:
len(validation_dataset)

1104

In [ ]:
validation_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 1104
})

In [ ]:
len(val_dataset2)

582

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering


In [ ]:
import collections



In [ ]:
import evaluate

metric = evaluate.load("squad")

In [ ]:
from tqdm.auto import tqdm


def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    "roberta-finetuned-subjqa-movies_2",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
import numpy as np
n_best=20
max_answer_length = 30

In [ ]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, val_dataset2)

  0%|          | 0/582 [00:00<?, ?it/s]

{'exact_match': 2.7491408934707904, 'f1': 9.852373992728378}

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.274200,No log
2,1.016300,No log
3,0.843000,No log
4,0.695900,No log
5,0.589200,No log


TrainOutput(global_step=3040, training_loss=0.8837257786800987, metrics={'train_runtime': 827.1503, 'train_samples_per_second': 29.39, 'train_steps_per_second': 3.675, 'total_flos': 4764093117189120.0, 'train_loss': 0.8837257786800987, 'epoch': 5.0})

In [ ]:
trainer.push_to_hub(commit_message="Training complete")

CommitInfo(commit_url='https://huggingface.co/mohamed13579/roberta-finetuned-subjqa-movies_2/commit/828e49a35b0df50a9b19f6faf4ab06438ea59970', commit_message='Training complete', commit_description='', oid='828e49a35b0df50a9b19f6faf4ab06438ea59970', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, val_dataset2)

  0%|          | 0/582 [00:00<?, ?it/s]

{'exact_match': 63.57388316151203, 'f1': 65.60935129342873}

In [ ]:
#Inference!

In [20]:
from transformers import pipeline

In [21]:
# Replace this with your own checkpoint
model_checkpoint2 = "mohamed13579/roberta-finetuned-subjqa-movies_2"
question_answerer = pipeline("question-answering", model=model_checkpoint2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [22]:
import pandas as pd
df_train1=pd.read_csv('/content/train.csv')
# df_test1=pd.read_csv('/content/test.csv')

In [23]:
df_train1.iloc[13].question

'Why is the movie soo confusing?'

In [24]:
df_train1.iloc[13].review

"Inception is an interesting movie but might not be everyone's cup of tea.  Dom Cobb (Leonardo DiCaprio) is a man who specializes in dream extractions (think corporate espionage) - going into a shared dream state using a military derived technique with a team and a subject to extract a piece of information from that subject's subconscious mind.  This can sometimes involve going into a dream within a dream (or more).Much was made of the complexities of this movie and indeed it is complex but for a generation growing up watching Matrix movies (especially the latter two) this is positively straight forward - and that's not a bad thing.  This is helped by Cobb's new dream architect (played by Ellen Page) who is also new to this world and to whom he explains the rules of the dream world (and thereby us - something not as well done in the Matrix flicks).  In addition to these dream rules there are the complexities of Cobb himself and the guilt he feels over the tragedy of his wife and kids a

In [25]:
context = df_train1.iloc[13].review
question = df_train1.iloc[13].question
question_answerer(question=question, context=context)

{'score': 0.27877935767173767,
 'start': 10,
 'end': 72,
 'answer': "is an interesting movie but might not be everyone's cup of tea"}

In [26]:
# Replace this with your own checkpoint
model_checkpoint_o = "deepset/roberta-base-squad2"
question_answerer_old = pipeline("question-answering", model=model_checkpoint_o)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [27]:
context = df_train1.iloc[13].review
question = df_train1.iloc[13].question
question_answerer_old(question=question, context=context)

{'score': 0.4546010494232178,
 'start': 2649,
 'end': 2669,
 'answer': 'they were so orderly'}

In [32]:
df_train.iloc[3].question

'Is this series good and excelent?'

In [34]:
df_train.iloc[3].answers

In [ ]:
df_train[['id','question','context','answers']].head()

,id,question,context,answers
0,0.0,Who is the author of this series?,Whether it be in her portrayal of a nerdy lesb...,"{'text': ['ANSWERNOTFOUND'], 'answer_start': [..."
1,1.0,Can we enjoy the movie along with our family ?,"An outstanding romantic comedy, 13 Going on 30...","{'text': ['ANSWERNOTFOUND'], 'answer_start': [..."
2,2.0,Does this one good?,"To let the truth be known, I watched this movi...","{'text': ['ANSWERNOTFOUND'], 'answer_start': [..."
3,3.0,Is this series good and excelent?,"At the time of my review, there had been 910 c...","{'text': ['this show is OUTSTANDING'], 'answer..."
4,4.0,How is the costume design?,"""Fright Night"" is great! This is how the story...",{'text': ['The costume design by Susan Matheso...


In [ ]:
len(df_train)

2501